### Лабораторная работа 2: основные понятия

В сегодняшней лабораторной будем работать с данными о проданных на аукционе автомобилях (`Carvana Car Prediction`). Целевым признаком здесь является наличие скрытых продавцом существенных недостатков. 

Carvana &mdash; компания-перекупщик автомобилей, соответственно, ее задача &mdash; не допустить скупку автомобилей со скрытыми недостатками, так как их впоследствии тяжелее продавать.

## 1

Загрузите данные о проданных на аукционе автомобилях. Посмотрите на "сырые" данные. 

Просмотрите файл `DataDictionary-ru.txt` и поймите, что означают столбцы матрицы. Выведите статистику по столбцам.


In [0]:
import pandas as pd

In [3]:
data = pd.read_csv('data.csv', sep=",", header=0)

FileNotFoundError: ignored

In [0]:
data.info()

## 2

Постройте график рассеяния с пробегом в качестве оси абсцисс и ценой `MMRCurrentRetailAveragePrice` в качестве оси ординат. Автомобили без недостатков отметьте зеленым цветом, со скрытыми недостатками — красным.

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
x=data[(data.IsBadBuy==1)].VehOdo
y=data[(data.IsBadBuy==1)].MMRCurrentRetailAveragePrice
plt.plot(x,y, color='red', ls="", marker="+")

In [0]:
x=data[(data.IsBadBuy==0)].VehOdo
y=data[(data.IsBadBuy==0)].MMRCurrentRetailAveragePrice
plt.plot(x,y, color='green',ls="", marker="+")

# Регрессия

## 3a

Импортируйте из библиотеки `sklearn` все модели машинного обучения:

`from sklearn import *`

Трактуя задачу о предсказании наличия недостатков как задачу регрессии, натренируйте линейную модель [`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) на каком-нибудь подмножестве признаков (например, тех же `VehOdo` и `MMRCurrentRetailAveragePrice`). 

С какой проблемой вы столкнулись? Какие способы ее решения вы можете предложить?

*Функции, которые могут пригодиться при решении: `.fit(), .loc[], pd.notnull()`*

*Создание модели:* `model1 = linear_model.LinearRegression()`

*Тренировка:*
`model1.fit(x, y)`



In [0]:
from sklearn import *

In [0]:
model1 = linear_model.LinearRegression()

In [0]:
data1=data[['VehOdo','VehicleAge','IsBadBuy']].dropna()

In [0]:
x=data1[['VehOdo' , 'VehicleAge']]

In [0]:
y=data1['IsBadBuy']

In [0]:
model1.fit(x,y)

## 3b

Выполните предсказание для всех объектов обучающей выборки и присвойте результат переменной prediction

*Функции, которые могут пригодиться при решении: `model.predict()`*

In [0]:
prediction=model1.predict(x)

## 3c

Преобразуйте получившийся вектор предсказаний `prediction` к значениям `{0,1}`. Это можно сделать, например, используя list comprehensions: https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions 

`predictionClass = [1 if prediction[i] > 0.5 else 0 for i in range(prediction.shape[0])]`

In [0]:
predictionClass1 = [1 if prediction[i] > 0.5 else 0 for i in range(prediction.shape[0])]
predictionClass2 = [1 if prediction[i] > 0.2 else 0 for i in range(prediction.shape[0])]

## 3d

Постройте отчет по качеству классификации и матрицу ошибок. Как изменятся отчет и матрица ошибок, если изменить порог в задании 3c (по умолчанию его значение равно 0.5)?

*Функции, которые могут пригодиться при решении: `print(metrics.classification_report(...))`, `print(metrics.confusion_matrix(...))`*

Функция classification_report для каждого класса объектов считает точность (precision) в этом классе и полноту (recall). Полнота - это процент объектов данного класса, которые ваш метод предсказания тоже отнес к этому классу, среди всех объектов данного класса. Точность (precision) - то же самое, только среди всех объектов, предсказанных для этого класса.

Функция confusion_matrix возвращает матрицу с количествами объектов. Номера столбцов матрицы - это номера предсказанных классов, строки - это номера правильных классов. Например, элемент M[0,1] - это количество машин, где на самом деле IsBadBuy = 0, а вы предсказали 1.

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report

In [0]:
print(classification_report (y,predictionClass1))
print(confusion_matrix (y, predictionClass1))

In [0]:
print(classification_report (y,predictionClass2))
print(confusion_matrix (y, predictionClass2))

# Классификация

## 4a
Трактуя задачу как задачу классификации, постройте модель классификации «решающее дерево» глубины 20  (все аналогично линейной регрессии).


*Функции, которые могут пригодиться при решении: `tree.DecisionTreeClassifier(max_depth=20) `*

In [0]:
tree1=tree.DecisionTreeClassifier(max_depth=20)
tree1.fit(x,y)
prediction_tree=tree1.predict(x)

## 4b

Решающее дерево предсказывает вероятности. С помощью построенной модели рассчитайте вероятности наличия скрываемых недостатков.

*Функции, которые могут пригодиться при решении: `model.predict_proba() `*

In [0]:
prediction_proba=tree1.predict_proba(x)

## 4c

Преобразуйте получившийся вектор предсказаний `prediction` к значениям `{0,1}`. Постройте отчет о классификации и матрицу ошибок. Какой метод оказался лучше?

In [0]:
predictionClass3 = [1 if prediction_tree[i] > 0.5 else 0 for i in range(prediction_tree.shape[0])]
print(classification_report (y,predictionClass3))
print(confusion_matrix (y, predictionClass3))

# Кросс-валидация
## 5a


Разделите исходную выборку на две части, например, используя [slices](https://pythonz.net/references/named/slice/):

`dataTrain = data.loc[0:34999,]
dataTest = data.loc[35000:69999,]`

In [0]:
from sklearn.model_selection import train_test_split
x_test, x_train, y_test, y_train=train_test_split(x,y,test_size=0.5)

## 5b

Натренируйте решающее дерево на  `dataTrain` и примените к  `dataTrain` и dataTest, вычислив для каждого случая точность предсказания. 

Проведите несколько экспериментов для различных глубин дерева. Напишите в комментариях, для каких глубин модель недообучена, для каких переобучена и где точка раннего останова.

*Функции, которые могут пригодиться при решении: `metrics.accuracy_score() `*

In [0]:
tree2=tree.DecisionTreeClassifier(max_depth=15)
tree2.fit(x_train,y_train)
prediction11=tree2.predict(x_train)
predictionClass4 = [1 if prediction11[i] > 0.5 else 0 for i in range(prediction11.shape[0])]
print(accuracy_score(y_train, predictionClass4))
print(accuracy_score(y_test,predictionClass4))

# Решающая функция
## 6a

Вернемся к модели решающего дерева глубины 20, построенного по всей обучающей выборке `data`. Пусть цена ошибки неправильного предсказания 0 равна 1000, а неправильного предсказания 1 — 100.

Вычислите функцию потерь &mdash; среднюю ошибку на всей обучающей выборке.

In [0]:
tree3=tree.DecisionTreeClassifier(max_depth=20)
tree3.fit(x_train,y_train)
prediction11=tree3.predict(x_train)
predictionClass4 = [1 if prediction11[i] > 0.5 else 0 for i in range(prediction11.shape[0])]
N=confusion_matrix (y_train, predictionClass4)
print(N[0,1]*100+N[1,0]*1000)

## 6b

Выясните, как нужно изменить решающую функцию `[0 if predictionProb[i][0] > 0.5 else 1 for i in range(prediction.shape[0])]`, чтобы функция потерь была минимальна?

Найдите оптимальную решающую функцию и минимальное значение функционала потерь методом подбора или пользуясь формулой из лекции 2 (слайд Decision function).



In [0]:
for q in range(0,100,3):
  print ("range ",q/100)
  tree3=tree.DecisionTreeClassifier(max_depth=20)
  tree3.fit(x_train,y_train)
  prediction11=tree3.predict(x_train)
  predictionClass4 = [1 if prediction11[i] > q/100 else 0 for i in range(prediction11.shape[0])]
  N=confusion_matrix (y_train, predictionClass4)
  print(N[0,1]*100+N[1,0]*1000)